## Installation and Imports
Please follow the **installation guide** in the [ThoughtSource Readme file](https://github.com/OpenBioLink/ThoughtSource) before using this notebook.

In [1]:
# only execute, if you use this notebook in Google Colab:
# !pip install -e ../libs/cot

In [44]:
import os
from cot import Collection
from cot.generate import FRAGMENTS
from rich.pretty import pprint
import json

## Overview
The ThoughtSource library offers functionality for: 
1) Loading datasets
2) Generating novel chain-of-thought reasoning data and answers
3) Evaluating results
4) Visualizing results on a Web Application

## 1. Loading, sampling and saving a dataset

In [3]:
# load a dataset to sample from 
worldtree = Collection(["worldtree"], verbose=False)
print(worldtree)

Loading worldtree...
| Name      |   Train |   Valid |   Test |
|-----------|---------|---------|--------|
| worldtree |    2207 |     496 |   1664 |

Not loaded: ['aqua', 'asdiv', 'commonsense_qa', 'entailment_bank', 'gsm8k', 'mawps', 'med_qa', 'medmc_qa', 'open_book_qa', 'pubmed_qa', 'qed', 'strategy_qa', 'svamp']


In [4]:
# Randomly select 100 rows from train split
worldtree_10 = worldtree.select(split="train", number_samples=10, random_samples=True, seed=0)
worldtree_10

| Name      |   Train | Valid   | Test   |
|-----------|---------|---------|--------|
| worldtree |      10 | -       | -      |

Not loaded: ['aqua', 'asdiv', 'commonsense_qa', 'entailment_bank', 'gsm8k', 'mawps', 'med_qa', 'medmc_qa', 'open_book_qa', 'pubmed_qa', 'qed', 'strategy_qa', 'svamp']

In [5]:
# Note that you could also sample from multiple datasets into one collection like this:
collection_medical = Collection(["med_qa", "medmc_qa", "pubmed_qa"], verbose=False)
collection_medical_100 = collection_medical.select(split="train", number_samples=100)
collection_medical_100

Loading med_qa...
Loading medmc_qa...
Loading pubmed_qa...


| Name      |   Train | Valid   | Test   |
|-----------|---------|---------|--------|
| med_qa    |     100 | -       | -      |
| medmc_qa  |     100 | -       | -      |
| pubmed_qa |     100 | -       | -      |

Not loaded: ['aqua', 'asdiv', 'commonsense_qa', 'entailment_bank', 'gsm8k', 'mawps', 'open_book_qa', 'qed', 'strategy_qa', 'svamp', 'worldtree']

## 2. Generating novel reasoning chains and answers

ThoughtSource comes pre-loaded with a large [collection of text snippets ('prompt fragments')](https://github.com/OpenBioLink/ThoughtSource/blob/main/libs/cot/cot/fragments.json) to elicit chain-of-thought reasoning in large language models and to extract answers from chains-of-thought. Let's see how prompt fragments look like:

In [6]:
# Chain of thought prompts
pprint(list(FRAGMENTS["cot_triggers"].items())[:5])

[
│   ('kojima-01', "Answer: Let's think step by step."),
│   ('kojima-02', 'Answer: We should think about this step by step.'),
│   ('kojima-03', 'Answer: First,'),
│   ('kojima-04', 'Answer: Before we dive into the answer,'),
│   ('kojima-05', 'Answer: Proof followed by the answer.')
]

In [7]:
# Answer extraction prompts
pprint(list(FRAGMENTS["answer_extractions"].items())[2:6])

[
│   ('kojima-03', 'The answer is'),
│   ('kojima-numerals', 'Therefore, the answer (arabic numerals) is'),
│   ('kojima-yes-no', 'Therefore, the answer (Yes or No) is'),
│   ('kojima-A-C', 'Therefore, among A through C, the answer is')
]

### Generating chain-of-thought examples

ThoughtSource can connect to external AI service providers such as the [OpenAI API](https://openai.com/api/) or the [Hugging Face Hub](https://huggingface.co/docs/hub/index). Set your token, 'api_service' and 'engine' parameters accordingly. 

In [49]:
from cot.config import Config as config_overview
print(r'\033[94m' + config_overview.__doc__[48:])


    "instruction_keys": list(str) - Determines which instruction_keys are used from fragments.json,
        the corresponding string will be inserted under "instruction" in the fragments. Default: [None] (No instruction)
    "cot_trigger_keys": list(str) - Determines which cot triggers are used from fragments.json,
        the corresponding string will be inserted under "cot_trigger" in the fragments. Default: ["kojima-01"]
    "answer_extraction_keys": list(str) - Determines which answer extraction prompts are used from fragments.json,
        the corresponding string will be inserted under "answer" in the fragments. Default: ["kojima-01"]
    "template_cot_generation": string - is the model input in the text generation step, variables in brackets.
        Only variables of this list are allowed: "instruction", 'question", "answer_choices", "cot_trigger"
        Default: {instruction}\n\n{question}\n{answer_choices}\n\n{cot_trigger}
    "template_answer_extraction": string - is the m

In [23]:
# Sample 100 items from the Worldtree v2 dataset
collection = Collection(["worldtree"], verbose=False)
worldtree_10 = collection.select(split="train", number_samples=10)

# Configuration for calling AI service. 
config={
    "instruction_keys": ['qa-01'], # "Answer the following question through step-by-step reasoning."
    "cot_trigger_keys": ['kojima-01'], # "Answer: Let's think step by step."
    "answer_extraction_keys": ['kojima-A-D'], # "Therefore, among A through D, the answer is"
    "author" : "your_name",
    "api_service": "mock_api", # <--- We use a mock API here for demonstration purposes of the tutorial.
    "engine": "", # 
    "temperature": 0,
    "max_tokens": 512,
    "verbose": False,
    "warn": True,
}

Loading worldtree...


In [24]:
#In the config, you can also define a custom text template.
#The default template for cot-generation is:

print("{instruction}\n\n{question}\n{answer_choices}\n\n{cot_trigger}")

{instruction}

{question}
{answer_choices}

{cot_trigger}


In [25]:
# You could also define your own template with a different structure and even free text.
print("Answer this question:\n{question}\n{answer_choices}\n\nGive a detailed explanation of your answer.")

Answer this question:
{question}
{answer_choices}

Give a detailed explanation of your answer.


In [26]:
# Generating chains-of-thought and answer extractions (This is in Mock-API mode, not calling model over API)
worldtree_10.generate(config=config) #if you cannot press y, set "warn" to false in config


        You are about to  call an external API  in total 20 times, which  may produce costs .
        Number API calls for CoT generation: n_samples 10 * n_instruction_keys 1 * n_cot_trigger_keys 1
        Number API calls for answer extraction: n_samples 10 * n_instruction_keys 1 * n_cot_trigger_keys 1 * n_answer_extraction_keys 1
        Do you want to continue? y/n
         Note: You are using a mock api. When entering 'y', a test run without API calls is made. 


The above was a fake call to the mock API
For the **purpose of the tutorial** we now load a prepared dataset with real model answers:

In [30]:
worldtree_10 = Collection.from_json("worldtree_10.json")

#### Display a question, answer choices and gold-standard answer

In [31]:
# Extract from prepared dataset
from pprint import pprint
pprint("Question: "+ worldtree_10["worldtree"]["train"][1]["question"])
pprint("Answer Options: " + ",  ".join(worldtree_10["worldtree"]["train"][1]["choices"]))
pprint("Gold Answer: "+ "".join(worldtree_10["worldtree"]["train"][1]["answer"]))

('Question: Animals may fight, make threatening sounds, and act aggressively '
 'toward members of the same species. These behaviors usually occur as the '
 'result of')
'Answer Options: competition,  conservation,  decomposition,  pollution'
'Gold Answer: competition'


#### Display model-generated chain-of-thought and extracted answer

In [32]:
pprint(worldtree_10["worldtree"]["train"][1]["generated_cot"][0]["cot"])
pprint(worldtree_10["worldtree"]["train"][1]["generated_cot"][0]['answers'][0]['answer'])

('Aggressive behaviors are often the result of competition. Competition is the '
 'result of animals fighting, making threatening sounds, and acting '
 'aggressively toward members of the same species. So, the final answer is A.')
'A.'


The answer generated by the model was correct! To evaluate model answers automatically, ThoughtSource has an in-built evaluate function.

In [40]:
# If you want to see your prompt as full text, you can use:
# This is not done by default as the json file gets very large.
worldtree_10.full_text_prompts()

  0%|          | 0/10 [00:00<?, ?ex/s]

In [37]:
# this is the exact text, which was used by the model to generate the chain of thought
print(worldtree_10["worldtree"]["train"][1]["generated_cot"][0]["prompt_text"])

Answer the following question through step-by-step reasoning.

Animals may fight, make threatening sounds, and act aggressively toward members of the same species. These behaviors usually occur as the result of
A) competition
B) conservation
C) decomposition
D) pollution

Answer: Let's think step by step.


## 3. Evaluate: Evaluation of model answers

In [38]:
worldtree_10.evaluate()

Evaluating worldtree train...


  0%|          | 0/10 [00:00<?, ?ex/s]

{'worldtree': {'train': {'accuracy': {'google/flan-t5-xl': {'qa-01_kojima-01_kojima-A-D': 0.6}}}}}

In [39]:
# Save the file that now also includes data in the 'correct_answer' fields 
worldtree_10.dump("worldtree_10.json")

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

## 4. Examine: Use the Web Tool to inspect the model outputs

### Here is the link: **[ThoughtSource Annotator](http://thought.samwald.info:3000/)**

Just **upload your just downloaded json file** to the web tool.



## Next step: Your own data creation with your personal API key.

ThoughtSource can connect to external AI service providers such as the [OpenAI API](https://openai.com/api/) or the [Hugging Face Hub](https://huggingface.co/docs/hub/index). Set your token, 'api_service' and 'engine' parameters accordingly.

In this tutorial we will use the Hugging Face Hub, which is for free. To use the API you need to set the environment variable `HUGGINGFACEHUB_API_TOKEN` to your API token. 

You can find your token in your Hugging Face settings page. You can set the environment variable in the following way:

In [ ]:
# os.environ["HUGGINGFACEHUB_API_TOKEN"] = "<token>"   # <--- set token (can be found in your Hugging Face settings page)
# os.environ["OPENAI_API_KEY"] = "<token>"  # <--- Set token for which API you want to use

#### The full process of loading, selecting, generating, evaluating and saving in one cell:

Calling the external API will approximately take 10 seconds

In [30]:
# 1) Dataset loading and selecting a random sample
collection = Collection(["worldtree"], verbose=False)
collection = collection.select(split="train", number_samples=1) # just using 1 sample for demonstration purposes

# 2) Language Model generates chains of thought and then extracts answers

config={
    "instruction_keys": ['qa-01'], # "Answer the following question through step-by-step reasoning."
    "cot_trigger_keys": ['kojima-01'], # "Answer: Let's think step by step."
    "answer_extraction_keys": ['kojima-A-D'], # "Therefore, among A through D, the answer is"
    "api_service": "huggingface_hub",
    "engine": "google/flan-t5-xl",
    "warn": False,
    "verbose": False,
}
collection.generate(config=config)

# 3) Evaluating answers generated by the model
print(collection.evaluate())

# 4) Saving the generated outputs and evaluation results
# collection.dump("worldtree_1.json")

# 5) Using the the ThoughSource Annotator web tool to inspect your results

Loading worldtree...
Evaluating worldtree train...
{'worldtree': {'train': {'accuracy': {'google/flan-t5-xl': {'qa-01_kojima-01_kojima-A-D': 1.0}}}}}
